In [2]:
import os
import time

import pandas as pd
import numpy as np
import xarray as xr

import scipy.stats as st

from itertools import combinations

from shapely.geometry import Point
import geopandas as gpd

from multiprocessing import Pool

/home/danbot/Documents/code/data_analysis/lib/python3.8/site-packages/geopandas/_compat.py:84: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.0-CAPI-1.16.2). Conversions between both will be slow.
  warnings.warn(


# Paired Watershed Characteristics

Develop framework to compare pairs of daily flow series from basins in the WSC database.  

## Method:

1. Generate a list of valid pairs of stations. A valid pair is one where:
    * basin geometry exists for both stations
    * there is a minimum N years of concurrent data between the two stations.

In [41]:
# import basin characteristics
WSC_db_folder = '/media/danbot/T7 Touch/hydat_db/'
metadata_fn = 'WSC_Stations_Master.csv'
hysets_folder = '/media/danbot/T7 Touch/hysets_series/'
camels_folder = '/media/danbot/T7 Touch/camels_db/usgs_streamflow/'

df = pd.read_csv(WSC_db_folder + metadata_fn)
df.head()

,Station Number,Station Name,Province,Status,Latitude,Longitude,Year From,Year To,Gross Drainage Area (km2),Effective Drainage Area (km2),...,Data Type,Operation Schedule,Sediment,RHBN,Real-Time,Datum Name,Publishing Office,Operating Agency,Contributed,Elevation
0,01AA002,DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE SHIDGEL,QC,Discontinued,46.55750,-70.08111,1967,1977,598.00,NaN,...,Flow,Continuous,N,N,N,ARBITRAIRE,QUEBEC CITY,MINISTERE DE L'ENVIRONNEMENT DU QUEBEC,Y,347.313904
1,01AA004,GAUTHIER (RIVIERE) A SON EMBOUCHURE,QC,Discontinued,46.80083,-70.13806,1975,1981,16.60,NaN,...,Level,Seasonal,N,N,N,ARBITRAIRE,QUEBEC CITY,MINISTERE DE L'ENVIRONNEMENT DU QUEBEC,Y,401.000000
2,01AB001,BUCKLEY (RIVIERE) EN AVAL DU LAC JOHNNY,QC,Discontinued,46.87917,-70.08750,1973,1981,6.94,NaN,...,Level,Seasonal,N,N,N,ARBITRAIRE,QUEBEC CITY,MINISTERE DE L'ENVIRONNEMENT DU QUEBEC,Y,411.000000
3,01AD001,MADAWASKA (RIVIÈRE) À 6 KM EN AVAL DU BARRAGE ...,QC,Discontinued,47.54833,-68.63639,1918,2005,2690.00,NaN,...,Flow,Continuous,N,N,N,ARBITRAIRE,QUEBEC CITY,MINISTERE DE L'ENVIRONNEMENT DU QUEBEC,Y,143.667770
4,01AD002,SAINT JOHN RIVER AT FORT KENT,NB,Active,47.25806,-68.59583,1926,2018,14700.00,NaN,...,Flow,Continuous,N,Y,N,INTERNATIONAL BOUNDARY COMMISSION DATUM,DARTMOUTH,UNITED STATES GEOLOGICAL SURVEY,Y,158.894211


In [9]:
df['num_years_record'] = df['Year To'] - df['Year From']

In [10]:
# filter for stations in BC and Alberta
df = df[df['Province'].isin(['BC', 'AB'])]
print(len(df))

3588


In [11]:
# create a list of all pairs of basins in the hydat database
hydat_stn_pairs_list = list(combinations(df['Station Number'].to_numpy(), 2))

print(len(df))
print(len(stn_pairs_list))

3588
6435078


In [95]:
# create a list of all camels files 
camels_files, camels_files_stations = [], []
for f in os.listdir(camels_folder):
    camels_files += [camels_folder + f + '/' + e for e in os.listdir(camels_folder + f)]
    camels_files_stations += [e.split('_')[0] for e in os.listdir(camels_folder + f)]


In [47]:
hysets_df = pd.read_csv('data/HYSETS_watershed_properties.txt', sep=';', dtype={'Official_ID': str})

In [48]:
# extract the HYDAT subset
hydat_df = hysets_df[hysets_df['Source'] == 'HYDAT'].copy()
# extract the USGS subset
usgs_df = hysets_df[hysets_df['Source'] == 'USGS'].copy()
# extract the Mexico subset
mex_df = hysets_df[hysets_df['Source'] == 'Mexico'].copy()

In [49]:
# create a centroid shapely Point
hydat_df['centroid_geom'] = hydat_df.apply(lambda xy: Point((xy['Centroid_Lon_deg_E'], xy['Centroid_Lat_deg_N'])), axis=1)
usgs_df['centroid_geom'] = usgs_df.apply(lambda xy: Point((xy['Centroid_Lon_deg_E'], xy['Centroid_Lat_deg_N'])), axis=1)
mex_df['centroid_geom'] = mex_df.apply(lambda xy: Point((xy['Centroid_Lon_deg_E'], xy['Centroid_Lat_deg_N'])), axis=1)

In [50]:
# find the common columns between all subsets of HYSETS basins
usgs_hydat_columns = np.intersect1d(hydat_df.columns, usgs_df.columns)
all_common_cols = np.intersect1d(usgs_hydat_columns, mex_df.columns)

In [51]:
# create a dictionary of identifying information to facilitate
# selection of specific watersheds
basin_metadata = ['Watershed_ID', 'Official_ID', 'Name']

basin_centroid_geom = ['centroid_geom']

basin_characteristics_cols = ['Centroid_Lat_deg_N', 'Centroid_Lon_deg_E',
              'Drainage_Area_GSIM_km2', 'Drainage_Area_km2',
              'Elevation_m', 'Gravelius', 'Perimeter', 'Slope_deg', 'Aspect_deg', 
              'Land_Use_Crops_frac', 'Land_Use_Forest_frac', 'Land_Use_Grass_frac',
              'Land_Use_Shrubs_frac', 'Land_Use_Snow_Ice_frac', 'Land_Use_Urban_frac',
              'Land_Use_Water_frac', 'Land_Use_Wetland_frac', 
              'Permeability_logk_m2', 'Porosity_frac']

In [119]:
hydat_dict = hydat_df[basin_metadata + basin_centroid_geom + basin_characteristics_cols].set_index('Official_ID').to_dict(orient='index')
usgs_dict = usgs_df[basin_metadata + basin_centroid_geom + basin_characteristics_cols].set_index('Official_ID').to_dict(orient='index')
mex_dict = mex_df[basin_metadata + basin_centroid_geom + basin_characteristics_cols].set_index('Official_ID').to_dict(orient='index')

In [120]:
# usgs_stns[:10]
# fix the station numbering zero padding convention in the usgs set from hysets
usgs_stns = []
for f in usgs_dict.keys():
    if len(f) == 8:
        usgs_stns.append(f)
    elif f[0] == '0':
        s = f[1:]
        if s[0] == '0':
            s = s[1:]
            if s[0] == '0':
                s = s[1:]
        if len(s) == 8:
            usgs_stns.append(s)
        

In [121]:
# find all station IDs common to HYSETS and CAMELS
camels_stns = np.intersect1d(usgs_stns, camels_files_stations)

In [122]:
hydat_stns, mex_stns = list(hydat_dict.keys()), list(mex_dict.keys())
# filter the usgs list for just the stations in CAMELS
print(f'There are {len(hydat_stns)} HYDAT station records in the HYSETS database.')
print(f'There are {len(camels_stns)} CAMELS station records in the HYSETS database.')
print(f'There are {len(mex_stns)} Mexico station records in the HYSETS database.')

There are 2375 HYDAT station records in the HYSETS database.
There are 645 CAMELS station records in the HYSETS database.
There are 46 Mexico station records in the HYSETS database.


In [123]:
hydat_stn_pairs_list = list(combinations(hydat_stns, 2))
camels_stn_pairs_list = list(combinations(camels_stns, 2))
mex_stn_pairs_list = list(combinations(mex_stns, 2))

In [124]:
hydat_pair_df = pd.DataFrame(hydat_stn_pairs_list, columns=['b1', 'b2'])
camels_pair_df = pd.DataFrame(camels_stn_pairs_list, columns=['b1', 'b2'])
mex_pair_df = pd.DataFrame(mex_stn_pairs_list, columns=['b1', 'b2'])

In [126]:
all_properties_dict = {'hydat': hydat_dict,
                      'camels': usgs_dict,
                       'mex': mex_dict,
                      }

In [136]:
def check_pair_properties(p, basin_dict):
    """
    Check if 
    """
    for c in basin_characteristics_cols:
        p1 = all_properties_dict[basin_dict][p['b1']][c]
        p2 = all_properties_dict[basin_dict][p['b2']][c]
    if ~np.isnan(p1) & ~np.isnan(p2):
        return True
    else:
        return False

In [138]:
mex_pair_df['char_check'] = mex_pair_df.apply(lambda row: check_pair_properties(row, 'mex'), axis=1)


In [139]:
camels_pair_df['char_check'] = camels_pair_df.apply(lambda row: check_pair_properties(row, 'camels'), axis=1)

In [140]:
hydat_pair_df['char_check'] = hydat_pair_df.apply(lambda row: check_pair_properties(row, 'hydat'), axis=1)

In [141]:
missing_characteristics = hydat_pair_df[~hydat_pair_df['char_check']].count()
print(f'{len(missing_characteristics)} basins in hydat have missing characteristics')
missing_characteristics = camels_pair_df[~camels_pair_df['char_check']].count()
print(f'{len(missing_characteristics)} basins in camels have missing characteristics')
missing_characteristics = mex_pair_df[~mex_pair_df['char_check']].count()
print(f'{len(missing_characteristics)} basins in Mexico have missing characteristics')

3 basins in hydat have missing characteristics
3 basins in camels have missing characteristics
3 basins in Mexico have missing characteristics


In [142]:
# filter out pairs missing basin characteristics
hydat_pair_df = hydat_pair_df[hydat_pair_df['char_check']]
camels_pair_df = camels_pair_df[camels_pair_df['char_check']]
mex_pair_df = mex_pair_df[mex_pair_df['char_check']]

In [143]:
def extract_streamflow_series(stn):
#     ws = hysets_dict[stn]
#     df = ds.sel(watershed=ws['Watershed_ID']-1, drop=True).to_dataframe()
    df = pd.read_csv(f'{hysets_folder}{stn}.csv', index_col=['time'])
    df.dropna(inplace=True)
    return df

In [144]:
def get_actual_concurrence_len(pair):
    df1 = extract_streamflow_series(pair[0])
    df1.rename(mapper={'discharge': f'{pair[0]}'}, inplace=True, axis=1)
    
    df2 = extract_streamflow_series(pair[1])
    df2.rename(mapper={'discharge': f'{pair[1]}'}, inplace=True, axis=1)
    concurrent_df = pd.concat([df1, df2], join='inner', axis=1)
    return len(concurrent_df)

In [147]:
t0 = time.time()
pool = Pool()

# pair_df['concurrence_check'] = pool.map(check_actual_concurrence_len, pair_df[['b1', 'b2']].to_numpy()[:10])
mex_pair_df['concurrent_days'] = pool.map(check_actual_concurrence_len, 
               mex_pair_df[['b1', 'b2']].to_numpy())
pool.close()
pool.join()
t1 = time.time()
print(f'Time to calculate concurrent period lengths: {t1 - t0:.1f}')

Time to calculate concurrent period lengths: 10.0


In [148]:
t0 = time.time()
pool = Pool()

# pair_df['concurrence_check'] = pool.map(check_actual_concurrence_len, pair_df[['b1', 'b2']].to_numpy()[:10])
camels_pair_df['concurrent_days'] = pool.map(check_actual_concurrence_len, 
               camels_pair_df[['b1', 'b2']].to_numpy())
pool.close()
pool.join()
t1 = time.time()
print(f'Time to calculate concurrent period lengths: {t1 - t0:.1f}')

Time to calculate concurrent period lengths: 2616.0


In [149]:
t0 = time.time()
pool = Pool()

# pair_df['concurrence_check'] = pool.map(check_actual_concurrence_len, pair_df[['b1', 'b2']].to_numpy()[:10])
hydat_pair_df['concurrent_days'] = pool.map(check_actual_concurrence_len, 
               hydat_pair_df[['b1', 'b2']].to_numpy())
pool.close()
pool.join()
t1 = time.time()
print(f'Time to calculate concurrent period lengths: {t1 - t0:.1f}')

Time to calculate concurrent period lengths: 12141.6


In [150]:
print(hydat_pair_df.head())
print(camels_pair_df.head())
print(mex_pair_df.head())
print(len(hydat_pair_df))
print(len(camels_pair_df))
print(len(mex_pair_df))


        b1       b2  char_check  concurrent_days
0  01AD002  01AD003        True            23103
1  01AD002  01AD015        True             1131
2  01AD002  01AE001        True            12418
4  01AD002  01AF007        True            13879
5  01AD002  01AF009        True             8766
         b1        b2  char_check  concurrent_days
0  01013500  01022500        True            25202
1  01013500  01030500        True            25202
2  01013500  01031500        True            25202
3  01013500  01047000        True            25202
4  01013500  01052500        True              806
      b1     b2  char_check  concurrent_days
0  10034  10063        True             2039
1  10034  11027        True             8729
2  10034  11040        True             8729
3  10034  12238        True            11284
4  10034  12370        True            13474
1826916
205761
1035


In [151]:
hydat_pair_df = hydat_pair_df[hydat_pair_df['concurrent_days'] > 365]
print(f'{len(hydat_pair_df)} HYDAT basin pairs meet the concurrence length criteria.')
camels_pair_df = camels_pair_df[camels_pair_df['concurrent_days'] > 365]
print(f'{len(camels_pair_df)} CAMELS basin pairs meet the concurrence length criteria.')
mex_pair_df = mex_pair_df[mex_pair_df['concurrent_days'] > 365]
print(f'{len(mex_pair_df)} Mexico basin pairs meet the concurrence length criteria.')

1436232 HYDAT basin pairs meet the concurrence length criteria.
205667 CAMELS basin pairs meet the concurrence length criteria.
969 Mexico basin pairs meet the concurrence length criteria.


In [152]:
# write the list of unique pairs to disk so you 
# don't have to go through that process again
hydat_pair_df.to_pickle('results/filtered_pairs_HYDAT_all_concurrent_lengths.csv')
camels_pair_df.to_pickle('results/filtered_pairs_CAMELS_all_concurrent_lengths.csv')
mex_pair_df.to_pickle('results/filtered_pairs_MEX_all_concurrent_lengths.csv')

In [153]:
filtered_pairs = camels_pair_df[['b1', 'b2']].to_numpy()
filtered_pairs = [tuple(e) for e in filtered_pairs]
unique_concurrent = list(set(filtered_pairs))
print(unique_concurrent[:5])
print(len(unique_concurrent))

[('01543500', '06910800'), ('02235200', '06614800'), ('03164000', '06406000'), ('03140000', '06224000'), ('02350900', '05488200')]
205667


## Re-Load all Saved Results

Continue the distance metric calculation

In [154]:
# all_df = pd.read_pickle('results/filtered_pairs_HYSETS_all_concurrent_lengths.csv')
all_df = pd.read_pickle('results/filtered_pairs_CAMELS_all_concurrent_lengths.csv')
all_df = all_df[['b1', 'b2', 'concurrent_days']]

## Run similarity operation on filtered pairs

1. Calculate a 'similarity' metric based on concurrent data.
2. Retrieve basin characteristics from the hysets basin characteristics file.
3. Calculate differences in basin elevation, gravelius, drainage area, and distance between basin centroids.

In [155]:
def get_param_diff(pair, param):
    return hysets_dict[pair[0]][param] - hysets_dict[pair[1]][param]

In [156]:
def get_distance(pair):
    foo = camels_df[hysets_df['Official_ID'].isin(pair)]
    hdf = gpd.GeoDataFrame(foo, geometry=foo['centroid_geom'], crs='EPSG:4326')
    hdf = hdf.to_crs(3005)
    hdf.reset_index(inplace=True)
    return hdf.loc[0, 'geometry'].distance(hdf.loc[1, 'geometry']) / 1000
    

In [159]:
import shapely.geometry as geom

def create_line(row):
    return geom.LineString([usgs_dict[row['b1']]['centroid_geom'], usgs_dict[row['b2']]['centroid_geom']])
    

In [160]:
# def set_centroid_geom()

# all_df['centroids'] = all_df.apply(row: Point(xy) for xy in )
geometry = gpd.GeoDataFrame({'geometry': all_df.apply(lambda row: create_line(row), axis=1)}, crs='EPSG:4326')

geometry = geometry.to_crs(3005)
geometry['centroid_distance_km'] = geometry.length / 1000  # convert to km
geometry.head()
# all_df.head()

# all_df['b1'].apply(lambda e: hysets_dict[e]['centroid_geom']) 
# all_df['b2_centroid_geom'] = all_df['b2'].apply(lambda e: hysets_dict[e]['centroid_geom']) 

,geometry,centroid_distance_km
0,"LINESTRING (4902150.132 1922495.212, 5148214.2...",294.705205
1,"LINESTRING (4902150.132 1922495.212, 5056283.6...",193.111443
2,"LINESTRING (4902150.132 1922495.212, 5027205.5...",234.857685
3,"LINESTRING (4902150.132 1922495.212, 5015698.1...",282.558269
4,"LINESTRING (4902150.132 1922495.212, 4952812.5...",324.124451


In [161]:
all_df['distance_btwn_centroids_km'] = geometry['centroid_distance_km']

all_df['pair_midpoint'] = geometry['geometry'].interpolate(0.5, normalized=False)

all_df.head()
# print(hysets_dict['05AA006'])

,b1,b2,concurrent_days,distance_btwn_centroids_km,pair_midpoint
0,01013500,01022500,25202,294.705205,POINT (4902150.549 1922494.936)
1,01013500,01030500,25202,193.111443,POINT (4902150.531 1922494.910)
2,01013500,01031500,25202,234.857685,POINT (4902150.398 1922494.788)
3,01013500,01047000,25202,282.558269,POINT (4902150.333 1922494.754)
4,01013500,01052500,806,324.124451,POINT (4902150.210 1922494.718)


In [162]:
# pair_df['normed_distance'] = (pair_df['PC_distance'] - pair_df['PC_distance'].min()) / (pair_df['PC_distance'].max() - pair_df['PC_distance'].min())

In [165]:
# foo = all_df.copy()
# convert back to EPSG 4326 for saving geographic coordinates
geometry = geometry.to_crs(4326)

all_df['midpoint_lat_deg_N'] = midpoint.apply(lambda mp: mp.y)
all_df['midpoint_lon_deg_E'] = midpoint.apply(lambda mp: mp.x)
all_df.head()

AttributeError: 'Series' object has no attribute 'y'

In [166]:
def calculate_similarity_measure_COD(pair):
    df1 = extract_streamflow_series(pair[0])    
    df2 = extract_streamflow_series(pair[1])

    concurrent_df = pd.concat([df1, df2], join='inner', axis=1)
    if len(concurrent_df) >= 365:
        cols = concurrent_df.columns
        out = st.linregress(concurrent_df.to_numpy())    

        return out[2]**2
    else:
        return np.nan
    

In [168]:
basin_characteristics_cols
for char in basin_characteristics_cols:
#     b1_char = 
    all_df[f'{char}_diff'] = [usgs_dict[b1][char] - usgs_dict[b2][char] for b1, b2 in all_df[['b1', 'b2']].to_numpy()]

In [169]:
pool = Pool()
t0 = time.time()
all_df['similarity'] = pool.map(calculate_similarity_measure_COD, all_df[['b1', 'b2']].to_numpy())
pool.close()
pool.join()
t1 = time.time()

In [170]:
print(f't for {len(all_df)} results: {t1-t0:.1f}s')
all_df.columns

t for 205667 results: 2724.2s


Index(['b1', 'b2', 'concurrent_days', 'distance_btwn_centroids_km',
       'pair_midpoint', 'Centroid_Lat_deg_N_diff', 'Centroid_Lon_deg_E_diff',
       'Drainage_Area_GSIM_km2_diff', 'Drainage_Area_km2_diff',
       'Elevation_m_diff', 'Gravelius_diff', 'Perimeter_diff',
       'Slope_deg_diff', 'Aspect_deg_diff', 'Land_Use_Crops_frac_diff',
       'Land_Use_Forest_frac_diff', 'Land_Use_Grass_frac_diff',
       'Land_Use_Shrubs_frac_diff', 'Land_Use_Snow_Ice_frac_diff',
       'Land_Use_Urban_frac_diff', 'Land_Use_Water_frac_diff',
       'Land_Use_Wetland_frac_diff', 'Permeability_logk_m2_diff',
       'Porosity_frac_diff', 'similarity'],
      dtype='object')

In [178]:
results_df = all_df.copy()
print(len(results_df))
results_df.dropna(subset=['similarity'], inplace=True)
print(len(results_df))
hydat_fp = 'results/results_HYDAT_min_365d_concurrent.csv'
camels_fp = 'results/results_CAMELS_min_365d_concurrent.csv'
mex_fp = 'results/results_MEX_min_365d_concurrent.csv'
results_df.to_csv(camels_fp, index=False)

205667
205667


In [179]:
import pandas as pd

foo = pd.read_csv('results/results_CAMELS_min_365d_concurrent.csv')

In [180]:
foo[foo['similarity'].isna()].count()

b1                             0
b2                             0
concurrent_days                0
distance_btwn_centroids_km     0
pair_midpoint                  0
Centroid_Lat_deg_N_diff        0
Centroid_Lon_deg_E_diff        0
Drainage_Area_GSIM_km2_diff    0
Drainage_Area_km2_diff         0
Elevation_m_diff               0
Gravelius_diff                 0
Perimeter_diff                 0
Slope_deg_diff                 0
Aspect_deg_diff                0
Land_Use_Crops_frac_diff       0
Land_Use_Forest_frac_diff      0
Land_Use_Grass_frac_diff       0
Land_Use_Shrubs_frac_diff      0
Land_Use_Snow_Ice_frac_diff    0
Land_Use_Urban_frac_diff       0
Land_Use_Water_frac_diff       0
Land_Use_Wetland_frac_diff     0
Permeability_logk_m2_diff      0
Porosity_frac_diff             0
similarity                     0
dtype: int64